# load-trace-deltas
Analyze the delta patterns present in a load trace, containing both VA and PAs.

In [1]:
import pandas as pd
from functools import partial

load_trace_path = '/scratch/cluster/cmolder/pythia/degree1_baseline/load_traces/libquantum_1210B_l2c.csv'

In [2]:
converters = {
    'address': partial(int, base=16),
    'phy_address': partial(int, base=16),
    'pc': partial(int, base=16),
}

load_trace = pd.read_csv(load_trace_path, converters=converters, index_col='instr_id')
load_trace.sort_index(inplace=True)
load_trace['address_line'] = (load_trace['address'].apply(lambda x: x >> 6))
load_trace['phy_address_line'] = (load_trace['phy_address'].apply(lambda x: x >> 6))
load_trace

,curr_cycle,address,phy_address,pc,cache_hit,address_line,phy_address_line
instr_id,,,,,,,
200000005,85279257,1039045056,3705831872,4202244,0,16235079,57903623
200000041,85279285,1039045120,3705831936,4202244,0,16235080,57903624
200000077,85279301,1039045184,3705832000,4202244,0,16235081,57903625
200000113,85279317,1039045248,3705832064,4202244,0,16235082,57903626
200000149,85279448,1039045312,3705832128,4202244,0,16235083,57903627
...,...,...,...,...,...,...,...
249999993,170191091,400370688,307522560,4202036,0,6255792,4805040
250000021,170191103,400370752,307522624,4202036,0,6255793,4805041
250000049,170191122,400370816,307522688,4202036,0,6255794,4805042


In [58]:
val_deltas = load_trace['address_line'].diff().dropna()
val_deltas = val_deltas.clip(-64, 64) # Clip large deltas
val_deltas.value_counts()

 1.0     1359334
 64.0      10879
-64.0      10718
Name: address_line, dtype: int64

In [59]:
pal_deltas = load_trace['phy_address_line'].diff().dropna()
pal_deltas = pal_deltas.clip(-64, 64) # Clip large deltas
pal_deltas.value_counts()

 1.0     1359334
-64.0      10868
 64.0      10729
Name: phy_address_line, dtype: int64